##### 우리 술 정보 api 불러오기

In [463]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pymongo import MongoClient

In [464]:
# 전통주 정보
url = 'https://api.odcloud.kr/api/15048755/v1/uddi:fec53d3a-2bef-4494-b50e-f4e566f403e0'
key = 'S256yGjpf7ug3eDtNRoJBzPLPIED9Mzp3XfMp8aaoRB/FGnDuzloLHkpvLvTyddzf00SKndA/1naWcmH2ao5jg=='
params = {'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1000', 'type' : 'json'}
response = requests.get(url, params=params)
per_page = response.json()['totalCount']
params = {'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1000','perPage': per_page, 'type' : 'json'}
response = requests.get(url, params=params)
data = pd.DataFrame(response.json()['data'])
sul_names = list(data['전통주명'])
    

In [465]:
sul_name [4]

'1'

##### 네이버 백과사전 링크 및 썸네일 가져오기

In [466]:
import urllib.request
import json
client_id = 'pLUBS8WQuLO7DTJbaxX0' # Your client_id
client_secret = 'YDFNUhy4T8' # Your client_secret
for sul_name in sul_names:
    query = sul_name
encText = urllib.parse.quote(query)
url = 'https://openapi.naver.com/v1/search/encyc?query=' + encText
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
response = urllib.request.urlopen(request)
data = response.read().decode('utf-8')
#print(data)
json_data = json.loads(data)
for items in json_data['items']:
    link = print(items['link'])
    thumbnail = print(items['thumbnail'])

https://terms.naver.com/entry.naver?docId=2849919&cid=51000&categoryId=51000
http://openapi-dbscthumb.phinf.naver.net/3323_000_25/20211107235431608_0LI5RPR65.jpg/P_2015072400014_.jpg?type=m160_160
https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B5%AD%EC%9D%98_%EC%86%8C%EC%A3%BC

https://en.wikipedia.org/wiki/Nick_Clegg

https://en.wikipedia.org/wiki/Greenpoint,_Brooklyn

https://en.wikipedia.org/wiki/Williamsburg,_Brooklyn

https://en.wikipedia.org/wiki/Randsburg,_California

https://en.wikipedia.org/wiki/People_of_Northern_Ireland

https://terms.naver.com/entry.naver?docId=5646455&cid=60406&categoryId=60406&anchorTarget=TABLE_OF_CONTENT15#TABLE_OF_CONTENT15

https://en.wikipedia.org/wiki/Culture_of_the_United_Kingdom

https://terms.naver.com/entry.naver?docId=2809609&cid=51000&categoryId=51000
http://openapi-dbscthumb.phinf.naver.net/3323_000_19/20201115235633635_1H0ADCFUE.jpg/P_2015062200003_.jpg?type=m160_160


#### 더술닷컴 크롤링
##### get_url = > go_next_page_url => get_data(url) => sqlite DB 밀어넣기 코드 

In [801]:
def get_page_url(self):
    page_url = []
    for i in range (71):
        num = str(i + 1)
        url = "https://thesool.com/front/find/M000000082/list.do?searchType=2&searchKey=&searchKind=&levelType=&searchString=&productId=&pageIndex="+num+"&categoryNm=&kind="
        page_url.append(url)
    return page_url

In [807]:
def get_product_url(page_url):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}  
    url_list = []
    for url in page_url:
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text,'html.parser')
        title_area = list(bs.find_all('div', {'class':'name'}))
        #url_list = []
        for i in title_area:
            find = int(str(i).index("("))
            find_2 = int(str(i).index(")"))
            num = str(i)[find+2:find_2-1]
            product_url = "https://thesool.com/front/find/M000000082/view.do?searchType=2&searchKey=&searchKind=&levelType=&searchString=&productId="+num+"&pageIndex=1&categoryNm=&kind="
            url_list.append(product_url)
    return url_list


In [301]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

def get_data(product_url_list):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
    all_data  = []    
    for url in product_url_list:
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text,'html.parser')
        data = []
        sul_name = bs.find('div', {'class':'product-view'}).find('dt', {'class':'subject'}).text
        sul_type = bs.find('div', {'class':'info'}).find('ul', {'class':'info-list'}).find('span').text.replace('\r','').replace('\n','').replace('\t','')
        sul_material = bs.find('div', {'class':'info'}).find('ul', {'class':'info-list'}).find_all('span')[1].text.replace('\r','').replace('\n','').replace('\t','')
        sul_alcohol = bs.find('div', {'class':'info'}).find('ul', {'class':'info-list'}).find_all('span')[2].text.replace('\r','').replace('\n','').replace('\t','').replace(' \xa0','')
        sul_capacity = bs.find('div', {'class':'info'}).find('ul', {'class':'info-list'}).find_all('span')[3].text.replace('\r','').replace('\n','').replace('\t','')
        sul_img = bs.find('div', {'class':'product-view'}).find('div', {'class':'img'}).find("img")["src"]
        sul_url = 'https://thesool.com'+sul_img
        try:
            sul_prize = bs.find('div', {'class':'info'}).find('ul', {'class':'info-list'}).find_all('span')[4].text.replace('\r','').replace('\n','').replace('\t','')
        except: None
        try:
            sul_etc = bs.find('div', {'class':'info'}).find('ul', {'class':'info-list'}).find_all('span')[5].text.replace('\r','').replace('\n','').replace('\t','')
        except:
            None
        sul_detail_info = bs.find('dd', {'class':'intro'}).find('div', {'class':'text'}).text.replace('\r','').replace('\n','').replace('\t','')
        sul_match_food = bs.find('dd', {'class':'food'}).find('div', {'class':'text'}).text.replace('\r','').replace('\n','').replace('\t','')
        distillery_name = bs.find('dd', {'class':'place'}).find('div', {'class':'text'}).find('span').text
        distillery_address = bs.find('dd', {'class':'place'}).find('div', {'class':'text'}).find_all('span')[1].text
        distillery_homepage = bs.find('dd', {'class':'place'}).find('div', {'class':'text'}).find_all('span')[2].text.replace('\r','').replace('\n','').replace('\t','')
        distillery_phone = bs.find('dd', {'class':'place'}).find('div', {'class':'text'}).find_all('li')[3].text.replace('문의','')
        data.append(sul_name)
        data.append(sul_type)
        data.append(sul_material)
        data.append(sul_alcohol)
        data.append(sul_capacity)
        data.append(sul_url)
        data.append(sul_prize)
        data.append(sul_etc)
        data.append(sul_detail_info)
        data.append(sul_match_food)
        data.append(distillery_name)
        data.append(distillery_address)
        data.append(distillery_homepage)
        data.append(distillery_phone)
        all_data.append(data)
    return all_data 

In [303]:
import pandas as pd

def convert_to_df(data):
     df = pd.DataFrame(data, columns = ['주종명','주종','재료','도수','용량','이미지', '수상내역','기타', '상세설명','페어링 음식','양조장_이름','양조장_주소','양조장_홈페이지','양조장_전화번호']) 
     return df  

In [546]:
import sqlite3
con = sqlite3.connect('yosul.db')
cur = con.cursor()
cur.execute("CREATE TABLE PhoneBook(Name text, PhoneNum text);")